In [7]:
import pandas as pd
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader, TensorDataset
from data import DivideData
import re
import glob

In [8]:
# check radar 114
path = '/home/Shared/xinyi/blob1/thesis/data/parquet_samples/14_06_22/radar_samples_192.168.67.114_289.parquet'
data = pd.read_parquet(path)
data.head()

In [4]:
check_data = pd.read_parquet('/home/Shared/xinyi/blob1/thesis/radar_112/all_part_tasks.parquet')
check_data

In [17]:
# Set the base path, subfolders, and task filename
base_dir = 'data/parquet_samples/'
file_pattern = os.path.join(base_dir, '**', 'radar_samples_192.168.67.112*')

matching_files = glob.glob(file_pattern, recursive=True)

In [21]:
matching_files

In [19]:
all_df = []
for file_path in matching_files:
    df = pd.read_parquet(file_path)
    all_df.append(df)
merged_df = pd.concat(all_df, ignore_index=True)

In [22]:
merged_df.to_parquet('radar_112/all_part_seg.parquet')

In [4]:
data1 = pd.read_parquet('radar_112/combined_task_all_0807.parquet')

In [9]:
data1['rx1_freq_a_channel_i_data'][0].shape

In [19]:
import os
import shutil

def move_files_to_root(root_folder):
    for dirpath, dirnames, filenames in os.walk(root_folder):
        if dirpath != root_folder:  # Skip the root folder itself
            for file in filenames:
                source_file = os.path.join(dirpath, file)
                destination_file = os.path.join(root_folder, file)
                
                # Move file to root folder
                shutil.move(source_file, destination_file)
                
            # After moving files, remove the empty directory
            os.rmdir(dirpath)

root_folder = "data/parquet_samples/30_06_22/"
move_files_to_root(root_folder)


In [43]:
import pandas as pd

data = pd.read_parquet('radar_112/combined_task_all_0207_2.parquet')
df = pd.DataFrame(data.iloc[:, :6])

max_length = max(max(len(arr) for arr in df[col]) for col in df.columns)

# Function to pad arrays to the maximum length
def pad_array(arr, max_length):
    padded = np.zeros(max_length)
    padded[:len(arr)] = arr
    return padded


# Pad each array in the DataFrame columns
padded_data = {col: [pad_array(arr, max_length) for arr in df[col]] for col in df.columns}

# Convert padded DataFrame columns to a single NumPy array
data_np = np.stack([np.vstack(padded_data[col]) for col in df.columns], axis=1)

# Convert NumPy array to PyTorch tensor
data_tensor = torch.tensor(data_np, dtype=torch.float64)

#data_np = np.stack([np.array(df[col].tolist()) for col in df.columns], axis=1)
# Convert DataFrame columns to a single NumPy array
#data_np = np.stack([np.vstack(df[col].values) for col in df.columns], axis=1)
# Convert DataFrame to a list of lists
#data_list = df.values.tolist()
data_tensor.shape

In [44]:
labels = data['label']
labels_tensor = torch.tensor(labels, dtype=torch.float32)
labels_tensor.shape

In [45]:
# Create the TensorDataset
dataset = TensorDataset(data_tensor, labels_tensor)

# Set the batch size
batch_size = 16

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [46]:
# Check the DataLoader
def check_dataloader(dataloader):
    for i, (data_batch, labels_batch) in enumerate(dataloader):
        print(f"Batch {i} data shape: {data_batch.shape}")
        print(f"Batch {i} labels shape: {labels_batch.shape}")
        print(f"Batch {i} data sample: {data_batch[0]}")
        print(f"Batch {i} labels sample: {labels_batch[0]}")
        if i >= 2:  # Limit to first 3 batches for brevity
            break
check_dataloader(dataloader)

In [9]:
# Transpose to get the shape (number of samples, number of columns, sequence length)
data_arrays = data_arrays.transpose(1, 0, 2)
data_arrays

In [16]:
import pandas as pd
import numpy as np
import os


radar_names = ['radar_52', 'radar_111', 'radar_112', 'radar_113', 'radar_114']

dates = ['14', '15', '16', '17', '20']
part_no = 14

for date in dates:
    for radar_name in radar_names:
        file_path = f'{date}_06/{radar_name}'
        # get the number of participants
        par_num = len(os.listdir(file_path))
        for j in range(1, par_num+1): 
            for i in range(0, 7):
                # Add label for each dataset, task and participant number
                data = pd.read_parquet(f"{file_path}/part_{j}/task_{i}.parquet")
                data['Task'] = i
                data['Participant'] = j + part_no
                data['Radar'] = radar_name
                data.to_parquet(f'{file_path}/part_{j}/task_{i}.parquet')
    part_no = part_no + par_num
    print(part_no)

In [23]:
path='20_06/radar_111/part_4/task_0.parquet'
data = pd.read_parquet(path)
data

In [45]:
radar_names = ['radar_112']
dates = ['14', '15', '16', '17', '20']


for date in dates:
    for radar_name in radar_names:
        # initiate the pd with data from 13_06_22
        radar_data = pd.read_parquet(f'13_06_22/{radar_name}/{radar_name}_data.parquet')
        # visit each participants
        file_path = f'{date}_06/{radar_name}'
        # get the number of participants
        par_num = len(os.listdir(file_path))
        for j in range(1, par_num+1): 
            for i in range(0, 7):
                data_new = pd.read_parquet(f"{file_path}/part_{j}/task_{i}.parquet")
                radar_data = pd.concat([radar_data, data_new], ignore_index=True)
        
        radar_data.to_parquet(f'{radar_name}_data_1405.parquet')



In [28]:
task_mapping = {
    'T_0': 0,
    'T_1': 1,
    'T_2': 2,
    'T_3': 3,
    'T_4': 4,
    'T_5': 5,
    'T_6': 6
}
radar_data['Task'] = radar_data['Task'].replace(task_mapping)  

In [34]:
participant_mapping = {
    f'P_{i}': i for i in range(1, 27)
}
radar_data['Participant'] = radar_data['Participant'].replace(participant_mapping)

In [42]:
change = pd.read_parquet('13_06_22/radar_112/radar_112_data.parquet')
task_mapping = {
    f'T_{i}': i for i in range(0, 7)
}
participant_mapping = {
    f'P_{i}': i for i in range(1, 5)
}
change['Task'] = change['Task'].replace(task_mapping)
change['Participant'] = change['Participant'].replace(participant_mapping)
change.to_parquet('13_06_22/radar_112/radar_112_data.parquet')

In [46]:
dada = pd.read_parquet('radar_112_data_1405.parquet')
dada

In [ ]:
# Group by the task column and concatenate the values of the first 6 columns
grouped = data.groupby('Task').agg(lambda x: x.tolist())
len(grouped['rx1_freq_a_channel_i_data']['T_0'])

In [ ]:
# Create a new DataFrame with the concatenated channel values
new_df = pd.DataFrame(grouped[['rx1_freq_a_channel_i_data', 'rx1_freq_a_channel_q_data',
                                'rx2_freq_a_channel_i_data', 'rx2_freq_a_channel_q_data',
                                'rx1_freq_b_channel_i_data', 'rx1_freq_b_channel_q_data']].values.tolist(),
                       columns=['rx1_freq_a_channel_i_data', 'rx1_freq_a_channel_q_data',
                                'rx2_freq_a_channel_i_data', 'rx2_freq_a_channel_q_data',
                                'rx1_freq_b_channel_i_data', 'rx1_freq_b_channel_q_data'])
new_df['Task'] = grouped.index
new_df.to_parquet('radar_114.parquet')

In [ ]:
dada = pd.read_parquet('radar_114.parquet')

In [ ]:
data, labels = dada.iloc[:, :6], dada['Task']

In [ ]:
task_l = data.values.tolist()

In [ ]:
def merge_and_pad(arrays, target_shape):
    # Create a new array filled with a padding value
    padded_array = np.zeros(target_shape, dtype=arrays[0].dtype)  # Assuming all arrays have the same dtype

    # Copy elements from each array into the new array, with padding
    for i, arr in enumerate(arrays):
        padded_array[i, :, :arr.shape[1]] += arr

    return padded_array

In [ ]:
task_arrays = [np.array(task_l[i]) for i in range(7)]
target_shape = (7, 6, 638976)
# Merge and pad the arrays
merged_array = merge_and_pad(task_arrays, target_shape)

print("Shape of merged array:", merged_array.shape)


In [ ]:
np.save('radar_114_task.npy', merged_array)

In [ ]:
import torch
data_tensor = torch.tensor(task_arr)

In [ ]:
class custom_dataset():
    def __init__(self, data=data_tensor, labels=labels):
        self.data = data
        self.labels = labels
        self.scaler = StandardScaler()  # You may need to scale the data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Each row in the DataFrame contains the signal data for one sample
        #sample = self.data.iloc[idx, :].values.astype(float)
        sample = self.data[idx]
        # Preprocess the data
        sample = self.scaler.fit_transform(sample.reshape(-1, 1))
        label = self.labels[idx]
        return sample, label

In [ ]:
dataset = custom_dataset(data_tensor, labels)

In [ ]:
# Create a DataLoader
batch_size = 64  # Adjust as needed
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
model = CNN()


In [ ]:
arr = grouped[['rx1_freq_a_channel_i_data', 'rx1_freq_a_channel_q_data',
               'rx2_freq_a_channel_i_data', 'rx2_freq_a_channel_q_data',
               'rx1_freq_b_channel_i_data', 'rx1_freq_b_channel_q_data']].values.tolist()

In [ ]:
import torch.nn as nn
m = nn.Linear(20, 30)
input = torch.randn(128, 20)
output = m(input)
print(output.size())

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [ ]:
path='13_06_22/radar_114/radar_114_data.parquet'
data = pd.read_parquet(path)

In [ ]:
training_set, test_set = train_test_split(data, test_size=0.10, random_state=42)

In [ ]:
training_set

In [ ]:
training_set.reset_index(drop=True, inplace=True)
test_set.reset_index(drop=True, inplace=True)

In [ ]:
test_set?

In [ ]:
col_names = ['rx1_freq_a_channel_i_data', 'rx1_freq_a_channel_q_data',
             'rx2_freq_a_channel_i_data', 'rx2_freq_a_channel_q_data',
             'rx1_freq_b_channel_i_data', 'rx1_freq_b_channel_q_data']


In [ ]:
# merge the signal with same task together
# group by the task column and concatenate the values of the first 6 columns
grouped_training = training_set.groupby('Task').agg(lambda x: x.tolist())
# create a new df with six channels and task, shape(7,6)
training_df = pd.DataFrame(grouped_training[col_names].values.tolist(), columns=col_names)
training_df['Task'] = grouped_training.index
training_df

In [ ]:
# same for test set
grouped_test = test_set.groupby('Task').agg(lambda x: x.tolist())
# create a new df with six channels and task, shape(7,6)
test_df = pd.DataFrame(grouped_test[col_names].values.tolist(), columns=col_names)
test_df['Task'] = grouped_test.index
test_df

In [ ]:
# get data and labels
training_data, training_labels = training_df.iloc[:, :6], training_df['Task']
test_data, test_labels = test_df.iloc[:, :6], test_df['Task']

In [ ]:
# convert data to list for array
training_task = training_data.values.tolist()
test_task = test_data.values.tolist()

In [ ]:
# padding and merge all task array together
training_task_arrays = [np.array(training_task[i]) for i in range(7)]
test_task_arrays = [np.array(test_task[i]) for i in range(7)]

In [ ]:
def merge_and_pad(arrays, target_shape):
    # Create a new array filled with a padding value
    padded_array = np.zeros(target_shape, dtype=arrays[0].dtype)  # Assuming all arrays have the same dtype

    # Copy elements from each array into the new array, with padding
    for i, arr in enumerate(arrays):
        padded_array[i, :, :arr.shape[1]] += arr

    return padded_array


In [ ]:
# set the target shape with 638976 after checking every task array, could be 6400000/480000 for all the data... good number^^
training_target_shape = (7, 6, 600000)
test_target_shape = (7, 6, 300000)

# Merge and pad the arrays
training_merged_array = merge_and_pad(training_task_arrays, training_target_shape)
test_merged_array = merge_and_pad(test_task_arrays, test_target_shape)


In [ ]:
# save as npy
np.save('radar_114_task_training.npy', training_merged_array)
np.save('radar_114_task_test.npy', test_merged_array)


In [ ]:
training_label_arr = np.array(training_labels)
test_label_arr = np.array(test_labels)

In [ ]:
np.save('radar_114_label_training.npy', training_label_arr)
np.save('radar_114_label_test.npy', test_label_arr)

In [ ]:
radar_name = 'radar_112'
data_path = f'{radar_name}_task_training.npy'
data_path